In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Sampler
from PIL import Image, ImageOps
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from resnest.torch import resnest50



import time
import pickle
import numpy as np
from torchvision.transforms import Lambda
import argparse
import copy
import random
import numbers
from torch.utils.tensorboard import SummaryWriter
from sklearn import metrics
import os

In [2]:
from NLBlock_MutiConv import NLBlock
from NLBlock_MutiConv import TimeConv

In [19]:
if torch.cuda.is_available()==True:
    device="cuda:2"
else:
    device ="cpu"
    
print(torch.cuda.is_available())

True


In [3]:
# Model 1  RCNet 
class M1_resnet_lstm(torch.nn.Module):
    def __init__(self):
        super(M1_resnet_lstm, self).__init__()
        resnet = models.resnet50(pretrained=True)
        self.share = torch.nn.Sequential()    # self.cnn = self.share
        self.share.add_module("conv1", resnet.conv1)
        self.share.add_module("bn1", resnet.bn1)
        self.share.add_module("relu", resnet.relu)
        self.share.add_module("maxpool", resnet.maxpool)
        self.share.add_module("layer1", resnet.layer1)
        self.share.add_module("layer2", resnet.layer2)
        self.share.add_module("layer3", resnet.layer3)
        self.share.add_module("layer4", resnet.layer4)
        self.share.add_module("avgpool", resnet.avgpool)
        self.dropout = nn.Dropout(p=0.2)
        self.lstm = nn.LSTM(2048, 512, batch_first=True) # feature : 512
        self.fc = nn.Linear(512, 19)      # 512 feature -> 19 classes


        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x = x.view(-1, 3, 216,216)  # 384 216 x.view(-1, 3, 224, 224) 
        x = self.share.forward(x)   # output [batchsize, 2048,1,1]
        x = x.view(-1, sequence_length, 2048) 
        
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)  # 512
        y = y.contiguous().view(-1, 512) # feature 
        y = self.dropout(y)
        y = self.fc(y)  # predict
        return y

In [7]:
MODEL = M1_resnet_lstm()
LFB_path = "/media/yilin/pre-trained models/LFB_model/LFB_resnet_sl10_M1.pth"

In [6]:
M1_10 = M1_resnet_lstm()
LFB_path = "/media/yilin/pre-trained models/LFB_model/LFB_resnet_sl10_M1.pth"
M1_10.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [12]:
M1_20 = M1_resnet_lstm()
LFB_path = "/media/yilin/pre-trained models/LFB_model/LFB_resnet_sl20_M1.pth"
M1_20.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [13]:
M1_30 = M1_resnet_lstm()
# LFB_path = "/media/yilin/pre-trained models/LFB_model/LFB_resnet_sl30_M1.pth"
LFB_path = "/media/yilin/tmrnet/models/temp/resnet/sl30_lr1e-5_6-25train/latest_model_10.pth"
M1_30.load_state_dict(torch.load(LFB_path))




<All keys matched successfully>

In [31]:
# # Model 4  resnet + lstm + non-local    
class M4_resnet_lstm_nl(torch.nn.Module):
    def __init__(self):
        super(M4_resnet_lstm_nl, self).__init__()
        resnet = models.resnet50(pretrained=True)
        self.share = torch.nn.Sequential()
        self.share.add_module("conv1", resnet.conv1)
        self.share.add_module("bn1", resnet.bn1)
        self.share.add_module("relu", resnet.relu)
        self.share.add_module("maxpool", resnet.maxpool)
        self.share.add_module("layer1", resnet.layer1)
        self.share.add_module("layer2", resnet.layer2)
        self.share.add_module("layer3", resnet.layer3)
        self.share.add_module("layer4", resnet.layer4)
        self.share.add_module("avgpool", resnet.avgpool)
        self.lstm = nn.LSTM(2048, 512, batch_first=True)
        self.fc_c = nn.Linear(512, 19) #7
        self.fc_h_c = nn.Linear(1024, 512)
        self.nl_block = NLBlock()
        self.dropout = nn.Dropout(p=0.5)

        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        init.xavier_uniform_(self.fc_c.weight)
        init.xavier_uniform_(self.fc_h_c.weight)

    def forward(self, x, long_feature=None):
        x = x.view(-1, 3, 216,216)
        x = self.share.forward(x)
        x = x.view(-1, sequence_length, 2048)
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        y = y[sequence_length - 1::sequence_length]

        y_1 = self.nl_block(y, long_feature)
        y = torch.cat([y, y_1], dim=1)
        y = self.dropout(self.fc_h_c(y))
        y = F.relu(y)
        y = self.fc_c(y)
        return y

In [32]:
M4 = M4_resnet_lstm_nl()
LFB_path = "/media/yilin/pre-trained models/CataRCNet_models/resnet_no-local-only_M4.pth"

In [33]:
M4.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [67]:
# # Model 4  resnet + lstm + non-local    
class M7_resnet_lstm_nl_tvl(torch.nn.Module):
    def __init__(self):
        super(M7_resnet_lstm_nl_tvl, self).__init__()
        resnet = models.resnet50(pretrained=True)
        self.share = torch.nn.Sequential()
        self.share.add_module("conv1", resnet.conv1)
        self.share.add_module("bn1", resnet.bn1)
        self.share.add_module("relu", resnet.relu)
        self.share.add_module("maxpool", resnet.maxpool)
        self.share.add_module("layer1", resnet.layer1)
        self.share.add_module("layer2", resnet.layer2)
        self.share.add_module("layer3", resnet.layer3)
        self.share.add_module("layer4", resnet.layer4)
        self.share.add_module("avgpool", resnet.avgpool)
        self.lstm = nn.LSTM(2048, 512, batch_first=True)
        self.fc_c = nn.Linear(512, 19) #7
        self.fc_h_c = nn.Linear(1024, 512)
        self.nl_block = NLBlock()
        self.dropout = nn.Dropout(p=0.5)
        self.time_conv = TimeConv()

        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        init.xavier_uniform_(self.fc_c.weight)
        init.xavier_uniform_(self.fc_h_c.weight)

    def forward(self, x, long_feature=None):
        x = x.view(-1, 3, 216,216)
        x = self.share.forward(x)
        x = x.view(-1, sequence_length, 2048)
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        y = y[sequence_length - 1::sequence_length]
        
        Lt = self.time_conv(long_feature)

        y_1 = self.nl_block(y, Lt)
        y = torch.cat([y, y_1], dim=1)
        y = self.dropout(self.fc_h_c(y))
        y = F.relu(y)
        y = self.fc_c(y)
        return y

In [68]:
M7 = M7_resnet_lstm_nl_tvl()
LFB_path = "/media/yilin/pre-trained models/CataRCNet_models/resnet_tvl_nl_M7.pth"

In [69]:
M7.load_state_dict(torch.load(LFB_path))

<All keys matched successfully>

In [59]:
# for M1 4 7
class LFB_resnet_lstm(torch.nn.Module):
    def __init__(self):
        super(LFB_resnet_lstm, self).__init__()
        resnet = models.resnet50(pretrained=True)
        self.share = torch.nn.Sequential()
        self.share.add_module("conv1", resnet.conv1)
        self.share.add_module("bn1", resnet.bn1)
        self.share.add_module("relu", resnet.relu)
        self.share.add_module("maxpool", resnet.maxpool)
        self.share.add_module("layer1", resnet.layer1)
        self.share.add_module("layer2", resnet.layer2)
        self.share.add_module("layer3", resnet.layer3)
        self.share.add_module("layer4", resnet.layer4)
        self.share.add_module("avgpool", resnet.avgpool)
        self.lstm = nn.LSTM(2048, 512, batch_first=True)

        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])

    def forward(self, x):
        x = x.view(-1, 3, 216,216)
        x = self.share.forward(x)
        x = x.view(-1, sequence_length, 2048)
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        y = y[sequence_length - 1::sequence_length]
        return y

# LOSS FUNCTION


In [4]:
class WeightedCrossEntropy(torch.nn.Module):
    '''
    WCE
    '''       
     # 6-25
    def __init__(self, weight=torch.Tensor([0.0033, 0.4182, 0.1321, 0.0234, 0.0344, 0.0146, 0.0428, 0.0140, 0.0092,
        0.0272, 0.0096, 0.0323, 0.0341, 0.0508, 0.0151, 0.0160, 0.0365, 0.0738,
        0.0128])):
        super(WeightedCrossEntropy, self).__init__()
        
        weight = weight.to(device)
        self.weighted_cross_entropy = nn.CrossEntropyLoss(weight=weight)
        
    def forward(self, inputs, target):
        return self.weighted_cross_entropy.forward(inputs, target)

In [5]:
def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

class CataractsDataset(Dataset):
    def __init__(self, file_paths,file_labels, transform=None,loader=pil_loader):
        self.file_paths = file_paths
        self.file_labels_phase = file_labels[:,0]
        self.transform = transform
        self.loader = loader
        
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        img_names = self.file_paths[index]
        labels = self.file_labels_phase[index]
        imgs = self.loader(img_names)
        if self.transform is not None:
            imgs = self.transform(imgs)

        return imgs, labels, index

In [6]:
######## center
def get_test_data(data_path):
    with open(data_path, 'rb') as f:
        test_paths_labels = pickle.load(f)

    test_paths = test_paths_labels[0]
    test_labels = test_paths_labels[1]
    test_num_each = test_paths_labels[2]

    print('test_paths   : {:6d}'.format(len(test_paths)))
    print('test_labels  : {:6d}'.format(len(test_labels)))

    test_labels = np.asarray(test_labels, dtype=np.int64)

    test_transforms = None
################################### 
    test_transforms = transforms.Compose([
            transforms.CenterCrop(216),  #transforms.RandomCrop(216),
            transforms.ToTensor()
        ])


    test_dataset = CataractsDataset(test_paths, test_labels, test_transforms)

    return test_dataset, test_num_each

# Test for M1

In [7]:
class SequenceSampler(Sampler):
    def __init__(self, data_source, idx):
        super().__init__(data_source)
        self.data_source = data_source
        self.idx = idx

    def __iter__(self):
        return iter(self.idx)

    def __len__(self):
        return len(self.idx)

In [8]:
# for sliding window
def get_start_idx(sequence_length, list_each_length):
    count = 0
    idx = []
    for i in range(len(list_each_length)):
        for j in range(count, count + (list_each_length[i] + 1 - sequence_length)):
            idx.append(j)
        count += list_each_length[i]
    return idx

In [9]:
def test_model(model,test_dataset, test_num_each):
    num_test = len(test_dataset)
    test_useful_start_idx = get_start_idx(sequence_length, test_num_each)
    test_useful_start_idx_LFB = get_start_idx(sequence_length, test_num_each)

    num_test_we_use = len(test_useful_start_idx)
    num_test_we_use_LFB = len(test_useful_start_idx_LFB)

    test_we_use_start_idx = test_useful_start_idx[0:num_test_we_use]
    test_we_use_start_idx_LFB = test_useful_start_idx_LFB[0:num_test_we_use_LFB]

    test_idx = []
    for i in range(num_test_we_use):
        for j in range(sequence_length):
            test_idx.append(test_we_use_start_idx[i] + j)

    num_test_all = len(test_idx)

    test_idx_LFB = []
    for i in range(num_test_we_use_LFB):
        for j in range(sequence_length):
            test_idx_LFB.append(test_we_use_start_idx_LFB[i] + j)

    dict_index, dict_value = zip(*list(enumerate(test_we_use_start_idx_LFB)))
    dict_test_start_idx_LFB = dict(zip(dict_value, dict_index))

    print('num test start idx : {:6d}'.format(len(test_useful_start_idx)))
    print('last idx test start: {:6d}'.format(test_useful_start_idx[-1]))
    print('num of test dataset: {:6d}'.format(num_test))
    print('num of test we use : {:6d}'.format(num_test_we_use))
    print('num of all test use: {:6d}'.format(num_test_all))
   # TODO sampler

    test_loader = DataLoader(test_dataset,
                             batch_size=test_batch_size,
                             sampler=SequenceSampler(test_dataset, test_idx),
                             num_workers=workers)

    
    torch.cuda.empty_cache()
    
    ###########################3
    # model = resnet_lstm()
    #model.load_state_dict(torch.load(model_path))
    model.load_state_dict(torch.load(LFB_path))
    
    ###############################
    

    model.to(device)

    criterion = WeightedCrossEntropy()   #size_average=False

    model.eval()
    test_loss = 0.0
    test_corrects = 0
    test_start_time = time.time()

    all_preds = []
    all_preds_score = []


    with torch.no_grad():

        for data in test_loader:
            

            torch.cuda.empty_cache()            


            inputs, labels = data[0].to(device), data[1].to(device)
            labels = labels[(sequence_length - 1)::sequence_length]


            inputs = inputs.view(-1, sequence_length, 3, 216,216)


            outputs = model.forward(inputs)

            outputs = outputs[sequence_length - 1::sequence_length]
            # softmax
            Sm = nn.Softmax()
            outputs = Sm(outputs)
            possibility, preds = torch.max(outputs.data, 1)


            for i in range(len(preds)):
                all_preds.append(preds[i].data.cpu())
            for i in range(len(possibility)):
                all_preds_score.append(possibility[i].data.cpu())
            
            
            loss = criterion(outputs, labels)


            test_loss += loss.data.item()
            test_corrects += torch.sum(preds == labels.data)
            print("test_corrects:",test_corrects)

    test_elapsed_time = time.time() - test_start_time
    test_accuracy = float(test_corrects) / float(num_test_we_use)
    test_average_loss = test_loss / num_test_we_use

    print('type of all_preds:', type(all_preds))
    print('leng of all preds:', len(all_preds))
    save_test = int("{:4.0f}".format(test_accuracy * 10000))
    pred_name = "./test_results_pkl/"+model_pure_name + '_test_' + str(save_test)  + '.pkl'
    

    with open(pred_name, 'wb') as f:
        pickle.dump(all_preds, f)

    print('test elapsed: {:2.0f}m{:2.0f}s'
          ' test loss: {:4.4f}'
          ' test accu: {:.4f}'
          .format(test_elapsed_time // 60,
                  test_elapsed_time % 60,
                  test_average_loss, test_accuracy))

In [10]:
# TEST25
test_dataset, test_num_each = get_test_data('../gen_datasets/test_paths_labels.pkl')

test_paths   :  15425
test_labels  :  15425


In [55]:

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2   
MODEL = M1_resnet_lstm() # M2_densenet_lstm() /  M3_resnest_lstm()


#########################################
device = "cuda:1"  

model_pure_name= 'LFB-resnet'
 # 'resnet/sl10_lr5e-5_6-25train/' cuda:0
LFB_path = "/media/yilin/pre-trained models/LFB_model/LFB_resnet_sl10_M1.pth"


# test M1
test_model(MODEL, test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000


/tmp/ipykernel_32289/935845069.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(10, device='cuda:1')
test_corrects: tensor(15, device='cuda:1')
test_corrects: tensor(15, device='cuda:1')
test_corrects: tensor(21, device='cuda:1')
test_corrects: tensor(29, device='cuda:1')
test_corrects: tensor(29, device='cuda:1')
test_corrects: tensor(38, device='cuda:1')
test_corrects: tensor(48, device='cuda:1')
test_corrects: tensor(55, device='cuda:1')
test_corrects: tensor(65, device='cuda:1')
test_corrects: tensor(71, device='cuda:1')
test_corrects: tensor(72, device='cuda:1')
test_corrects: tensor(72, device='cuda:1')
test_corrects: tensor(75, device='cuda:1')
test_corrects: tensor(75, device='cuda:1')
test_corrects: tensor(75, device='cuda:1')
test_corrects: tensor(82, device='cuda:1')
test_corrects: tensor(82, device='cuda:1')
test_corrects: tensor(84, device='cuda:1')
test_corrects: tensor(89, device='cuda:1')
test_corrects: tensor(95, device='cuda:1')
test_corrects: tensor(104, device='cuda:1')
test_corrects: tensor(114, device='cuda:1')
test_corr

test_corrects: tensor(1057, device='cuda:1')
test_corrects: tensor(1066, device='cuda:1')
test_corrects: tensor(1076, device='cuda:1')
test_corrects: tensor(1081, device='cuda:1')
test_corrects: tensor(1081, device='cuda:1')
test_corrects: tensor(1087, device='cuda:1')
test_corrects: tensor(1097, device='cuda:1')
test_corrects: tensor(1106, device='cuda:1')
test_corrects: tensor(1107, device='cuda:1')
test_corrects: tensor(1110, device='cuda:1')
test_corrects: tensor(1113, device='cuda:1')
test_corrects: tensor(1118, device='cuda:1')
test_corrects: tensor(1118, device='cuda:1')
test_corrects: tensor(1125, device='cuda:1')
test_corrects: tensor(1134, device='cuda:1')
test_corrects: tensor(1144, device='cuda:1')
test_corrects: tensor(1154, device='cuda:1')
test_corrects: tensor(1164, device='cuda:1')
test_corrects: tensor(1165, device='cuda:1')
test_corrects: tensor(1165, device='cuda:1')
test_corrects: tensor(1169, device='cuda:1')
test_corrects: tensor(1169, device='cuda:1')
test_corre

test_corrects: tensor(2342, device='cuda:1')
test_corrects: tensor(2352, device='cuda:1')
test_corrects: tensor(2361, device='cuda:1')
test_corrects: tensor(2370, device='cuda:1')
test_corrects: tensor(2380, device='cuda:1')
test_corrects: tensor(2389, device='cuda:1')
test_corrects: tensor(2398, device='cuda:1')
test_corrects: tensor(2408, device='cuda:1')
test_corrects: tensor(2418, device='cuda:1')
test_corrects: tensor(2428, device='cuda:1')
test_corrects: tensor(2438, device='cuda:1')
test_corrects: tensor(2448, device='cuda:1')
test_corrects: tensor(2454, device='cuda:1')
test_corrects: tensor(2464, device='cuda:1')
test_corrects: tensor(2474, device='cuda:1')
test_corrects: tensor(2484, device='cuda:1')
test_corrects: tensor(2494, device='cuda:1')
test_corrects: tensor(2504, device='cuda:1')
test_corrects: tensor(2513, device='cuda:1')
test_corrects: tensor(2521, device='cuda:1')
test_corrects: tensor(2530, device='cuda:1')
test_corrects: tensor(2540, device='cuda:1')
test_corre

...

test_corrects: tensor(5148, device='cuda:1')
test_corrects: tensor(5150, device='cuda:1')
test_corrects: tensor(5159, device='cuda:1')
test_corrects: tensor(5169, device='cuda:1')
test_corrects: tensor(5174, device='cuda:1')
test_corrects: tensor(5174, device='cuda:1')
test_corrects: tensor(5184, device='cuda:1')
test_corrects: tensor(5193, device='cuda:1')
test_corrects: tensor(5202, device='cuda:1')
test_corrects: tensor(5208, device='cuda:1')
test_corrects: tensor(5217, device='cuda:1')
test_corrects: tensor(5227, device='cuda:1')
test_corrects: tensor(5237, device='cuda:1')
test_corrects: tensor(5247, device='cuda:1')
test_corrects: tensor(5257, device='cuda:1')
test_corrects: tensor(5267, device='cuda:1')
test_corrects: tensor(5277, device='cuda:1')
test_corrects: tensor(5287, device='cuda:1')
test_corrects: tensor(5297, device='cuda:1')
test_corrects: tensor(5306, device='cuda:1')
test_corrects: tensor(5315, device='cuda:1')
test_corrects: tensor(5318, device='cuda:1')
test_corre

test_corrects: tensor(6546, device='cuda:1')
test_corrects: tensor(6549, device='cuda:1')
test_corrects: tensor(6551, device='cuda:1')
test_corrects: tensor(6561, device='cuda:1')
test_corrects: tensor(6571, device='cuda:1')
test_corrects: tensor(6581, device='cuda:1')
test_corrects: tensor(6591, device='cuda:1')
test_corrects: tensor(6601, device='cuda:1')
test_corrects: tensor(6611, device='cuda:1')
test_corrects: tensor(6621, device='cuda:1')
test_corrects: tensor(6630, device='cuda:1')
test_corrects: tensor(6638, device='cuda:1')
test_corrects: tensor(6644, device='cuda:1')
test_corrects: tensor(6654, device='cuda:1')
test_corrects: tensor(6663, device='cuda:1')
test_corrects: tensor(6673, device='cuda:1')
test_corrects: tensor(6683, device='cuda:1')
test_corrects: tensor(6693, device='cuda:1')
test_corrects: tensor(6703, device='cuda:1')
test_corrects: tensor(6713, device='cuda:1')
test_corrects: tensor(6723, device='cuda:1')
test_corrects: tensor(6732, device='cuda:1')
test_corre

test_corrects: tensor(7957, device='cuda:1')
test_corrects: tensor(7967, device='cuda:1')
test_corrects: tensor(7977, device='cuda:1')
test_corrects: tensor(7987, device='cuda:1')
test_corrects: tensor(7993, device='cuda:1')
test_corrects: tensor(7995, device='cuda:1')
test_corrects: tensor(7995, device='cuda:1')
test_corrects: tensor(7995, device='cuda:1')
test_corrects: tensor(8000, device='cuda:1')
test_corrects: tensor(8010, device='cuda:1')
test_corrects: tensor(8020, device='cuda:1')
test_corrects: tensor(8030, device='cuda:1')
test_corrects: tensor(8040, device='cuda:1')
test_corrects: tensor(8050, device='cuda:1')
test_corrects: tensor(8059, device='cuda:1')
test_corrects: tensor(8069, device='cuda:1')
test_corrects: tensor(8078, device='cuda:1')
test_corrects: tensor(8085, device='cuda:1')
test_corrects: tensor(8095, device='cuda:1')
test_corrects: tensor(8102, device='cuda:1')
test_corrects: tensor(8105, device='cuda:1')
test_corrects: tensor(8112, device='cuda:1')
test_corre

test_corrects: tensor(9225, device='cuda:1')
test_corrects: tensor(9235, device='cuda:1')
test_corrects: tensor(9245, device='cuda:1')
test_corrects: tensor(9255, device='cuda:1')
test_corrects: tensor(9265, device='cuda:1')
test_corrects: tensor(9272, device='cuda:1')
test_corrects: tensor(9281, device='cuda:1')
test_corrects: tensor(9291, device='cuda:1')
test_corrects: tensor(9300, device='cuda:1')
test_corrects: tensor(9310, device='cuda:1')
test_corrects: tensor(9320, device='cuda:1')
test_corrects: tensor(9329, device='cuda:1')
test_corrects: tensor(9335, device='cuda:1')
test_corrects: tensor(9338, device='cuda:1')
test_corrects: tensor(9343, device='cuda:1')
test_corrects: tensor(9353, device='cuda:1')
test_corrects: tensor(9362, device='cuda:1')
test_corrects: tensor(9367, device='cuda:1')
test_corrects: tensor(9372, device='cuda:1')
test_corrects: tensor(9376, device='cuda:1')
test_corrects: tensor(9386, device='cuda:1')
test_corrects: tensor(9396, device='cuda:1')
test_corre

test_corrects: tensor(10766, device='cuda:1')
test_corrects: tensor(10774, device='cuda:1')
test_corrects: tensor(10779, device='cuda:1')
test_corrects: tensor(10785, device='cuda:1')
test_corrects: tensor(10793, device='cuda:1')
test_corrects: tensor(10797, device='cuda:1')
test_corrects: tensor(10802, device='cuda:1')
test_corrects: tensor(10810, device='cuda:1')
test_corrects: tensor(10811, device='cuda:1')
test_corrects: tensor(10814, device='cuda:1')
test_corrects: tensor(10824, device='cuda:1')
test_corrects: tensor(10834, device='cuda:1')
test_corrects: tensor(10844, device='cuda:1')
test_corrects: tensor(10854, device='cuda:1')
test_corrects: tensor(10862, device='cuda:1')
test_corrects: tensor(10863, device='cuda:1')
test_corrects: tensor(10863, device='cuda:1')
test_corrects: tensor(10867, device='cuda:1')
test_corrects: tensor(10877, device='cuda:1')
test_corrects: tensor(10887, device='cuda:1')
test_corrects: tensor(10897, device='cuda:1')
test_corrects: tensor(10907, devic

In [23]:
# example output of 2 test videos, 
# if you cannot load the pretrained model, you can change the layer name self.cnn to self.share

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2   
MODEL = M1_resnet_lstm() # M2_densenet_lstm() /  M3_resnest_lstm()


#########################################
device = "cuda:1"  

model_pure_name= 'LFB-resnet-20'
 # 'resnet/sl10_lr5e-5_6-25train/' cuda:0
LFB_path = "/media/yilin/pre-trained models/LFB_model/LFB_resnet_sl20_M1.pth"


# test M1
test_model(MODEL, test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000


/tmp/ipykernel_27798/935845069.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(7, device='cuda:1')
test_corrects: tensor(13, device='cuda:1')
test_corrects: tensor(23, device='cuda:1')
test_corrects: tensor(30, device='cuda:1')
test_corrects: tensor(38, device='cuda:1')
test_corrects: tensor(42, device='cuda:1')
test_corrects: tensor(52, device='cuda:1')
test_corrects: tensor(62, device='cuda:1')
test_corrects: tensor(71, device='cuda:1')
test_corrects: tensor(71, device='cuda:1')
test_corrects: tensor(73, device='cuda:1')
test_corrects: tensor(82, device='cuda:1')
test_corrects: tensor(92, device='cuda:1')
test_corrects: tensor(95, device='cuda:1')
test_corrects: tensor(95, device='cuda:1')
test_corrects: tensor(95, device='cuda:1')
test_corrects: tensor(103, device='cuda:1')
test_corrects: tensor(113, device='cuda:1')
test_corrects: tensor(123, device='cuda:1')
test_corrects: tensor(133, device='cuda:1')
test_corrects: tensor(143, device='cuda:1')
test_corrects: tensor(153, device='cuda:1')
test_corrects: tensor(163, device='cuda:1')
test_

test_corrects: tensor(1121, device='cuda:1')
test_corrects: tensor(1131, device='cuda:1')
test_corrects: tensor(1140, device='cuda:1')
test_corrects: tensor(1142, device='cuda:1')
test_corrects: tensor(1142, device='cuda:1')
test_corrects: tensor(1148, device='cuda:1')
test_corrects: tensor(1158, device='cuda:1')
test_corrects: tensor(1167, device='cuda:1')
test_corrects: tensor(1175, device='cuda:1')
test_corrects: tensor(1177, device='cuda:1')
test_corrects: tensor(1181, device='cuda:1')
test_corrects: tensor(1189, device='cuda:1')
test_corrects: tensor(1189, device='cuda:1')
test_corrects: tensor(1199, device='cuda:1')
test_corrects: tensor(1209, device='cuda:1')
test_corrects: tensor(1219, device='cuda:1')
test_corrects: tensor(1229, device='cuda:1')
test_corrects: tensor(1239, device='cuda:1')
test_corrects: tensor(1249, device='cuda:1')
test_corrects: tensor(1259, device='cuda:1')
test_corrects: tensor(1269, device='cuda:1')
test_corrects: tensor(1279, device='cuda:1')
test_corre

test_corrects: tensor(2436, device='cuda:1')
test_corrects: tensor(2446, device='cuda:1')
test_corrects: tensor(2456, device='cuda:1')
test_corrects: tensor(2464, device='cuda:1')
test_corrects: tensor(2474, device='cuda:1')
test_corrects: tensor(2484, device='cuda:1')
test_corrects: tensor(2494, device='cuda:1')
test_corrects: tensor(2504, device='cuda:1')
test_corrects: tensor(2514, device='cuda:1')
test_corrects: tensor(2524, device='cuda:1')
test_corrects: tensor(2534, device='cuda:1')
test_corrects: tensor(2544, device='cuda:1')
test_corrects: tensor(2549, device='cuda:1')
test_corrects: tensor(2559, device='cuda:1')
test_corrects: tensor(2569, device='cuda:1')
test_corrects: tensor(2579, device='cuda:1')
test_corrects: tensor(2589, device='cuda:1')
test_corrects: tensor(2598, device='cuda:1')
test_corrects: tensor(2608, device='cuda:1')
test_corrects: tensor(2616, device='cuda:1')
test_corrects: tensor(2625, device='cuda:1')
test_corrects: tensor(2635, device='cuda:1')
test_corre

...

test_corrects: tensor(5118, device='cuda:1')
test_corrects: tensor(5121, device='cuda:1')
test_corrects: tensor(5130, device='cuda:1')
test_corrects: tensor(5140, device='cuda:1')
test_corrects: tensor(5145, device='cuda:1')
test_corrects: tensor(5145, device='cuda:1')
test_corrects: tensor(5153, device='cuda:1')
test_corrects: tensor(5163, device='cuda:1')
test_corrects: tensor(5172, device='cuda:1')
test_corrects: tensor(5176, device='cuda:1')
test_corrects: tensor(5184, device='cuda:1')
test_corrects: tensor(5194, device='cuda:1')
test_corrects: tensor(5204, device='cuda:1')
test_corrects: tensor(5214, device='cuda:1')
test_corrects: tensor(5224, device='cuda:1')
test_corrects: tensor(5234, device='cuda:1')
test_corrects: tensor(5244, device='cuda:1')
test_corrects: tensor(5254, device='cuda:1')
test_corrects: tensor(5264, device='cuda:1')
test_corrects: tensor(5272, device='cuda:1')
test_corrects: tensor(5278, device='cuda:1')
test_corrects: tensor(5287, device='cuda:1')
test_corre

test_corrects: tensor(6561, device='cuda:1')
test_corrects: tensor(6561, device='cuda:1')
test_corrects: tensor(6563, device='cuda:1')
test_corrects: tensor(6573, device='cuda:1')
test_corrects: tensor(6583, device='cuda:1')
test_corrects: tensor(6593, device='cuda:1')
test_corrects: tensor(6603, device='cuda:1')
test_corrects: tensor(6613, device='cuda:1')
test_corrects: tensor(6623, device='cuda:1')
test_corrects: tensor(6633, device='cuda:1')
test_corrects: tensor(6642, device='cuda:1')
test_corrects: tensor(6642, device='cuda:1')
test_corrects: tensor(6648, device='cuda:1')
test_corrects: tensor(6658, device='cuda:1')
test_corrects: tensor(6668, device='cuda:1')
test_corrects: tensor(6678, device='cuda:1')
test_corrects: tensor(6688, device='cuda:1')
test_corrects: tensor(6698, device='cuda:1')
test_corrects: tensor(6708, device='cuda:1')
test_corrects: tensor(6718, device='cuda:1')
test_corrects: tensor(6728, device='cuda:1')
test_corrects: tensor(6735, device='cuda:1')
test_corre

test_corrects: tensor(7960, device='cuda:1')
test_corrects: tensor(7970, device='cuda:1')
test_corrects: tensor(7980, device='cuda:1')
test_corrects: tensor(7990, device='cuda:1')
test_corrects: tensor(8000, device='cuda:1')
test_corrects: tensor(8007, device='cuda:1')
test_corrects: tensor(8011, device='cuda:1')
test_corrects: tensor(8011, device='cuda:1')
test_corrects: tensor(8011, device='cuda:1')
test_corrects: tensor(8015, device='cuda:1')
test_corrects: tensor(8025, device='cuda:1')
test_corrects: tensor(8035, device='cuda:1')
test_corrects: tensor(8045, device='cuda:1')
test_corrects: tensor(8055, device='cuda:1')
test_corrects: tensor(8063, device='cuda:1')
test_corrects: tensor(8072, device='cuda:1')
test_corrects: tensor(8081, device='cuda:1')
test_corrects: tensor(8089, device='cuda:1')
test_corrects: tensor(8098, device='cuda:1')
test_corrects: tensor(8108, device='cuda:1')
test_corrects: tensor(8118, device='cuda:1')
test_corrects: tensor(8125, device='cuda:1')
test_corre

test_corrects: tensor(9181, device='cuda:1')
test_corrects: tensor(9186, device='cuda:1')
test_corrects: tensor(9195, device='cuda:1')
test_corrects: tensor(9205, device='cuda:1')
test_corrects: tensor(9215, device='cuda:1')
test_corrects: tensor(9225, device='cuda:1')
test_corrects: tensor(9235, device='cuda:1')
test_corrects: tensor(9244, device='cuda:1')
test_corrects: tensor(9254, device='cuda:1')
test_corrects: tensor(9264, device='cuda:1')
test_corrects: tensor(9274, device='cuda:1')
test_corrects: tensor(9284, device='cuda:1')
test_corrects: tensor(9294, device='cuda:1')
test_corrects: tensor(9303, device='cuda:1')
test_corrects: tensor(9309, device='cuda:1')
test_corrects: tensor(9314, device='cuda:1')
test_corrects: tensor(9318, device='cuda:1')
test_corrects: tensor(9328, device='cuda:1')
test_corrects: tensor(9338, device='cuda:1')
test_corrects: tensor(9347, device='cuda:1')
test_corrects: tensor(9357, device='cuda:1')
test_corrects: tensor(9361, device='cuda:1')
test_corre

test_corrects: tensor(10717, device='cuda:1')
test_corrects: tensor(10727, device='cuda:1')
test_corrects: tensor(10736, device='cuda:1')
test_corrects: tensor(10744, device='cuda:1')
test_corrects: tensor(10754, device='cuda:1')
test_corrects: tensor(10759, device='cuda:1')
test_corrects: tensor(10765, device='cuda:1')
test_corrects: tensor(10773, device='cuda:1')
test_corrects: tensor(10777, device='cuda:1')
test_corrects: tensor(10781, device='cuda:1')
test_corrects: tensor(10791, device='cuda:1')
test_corrects: tensor(10798, device='cuda:1')
test_corrects: tensor(10803, device='cuda:1')
test_corrects: tensor(10813, device='cuda:1')
test_corrects: tensor(10823, device='cuda:1')
test_corrects: tensor(10833, device='cuda:1')
test_corrects: tensor(10843, device='cuda:1')
test_corrects: tensor(10851, device='cuda:1')
test_corrects: tensor(10851, device='cuda:1')
test_corrects: tensor(10851, device='cuda:1')
test_corrects: tensor(10853, device='cuda:1')
test_corrects: tensor(10863, devic

In [29]:

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2   
MODEL = M1_resnet_lstm() # M2_densenet_lstm() /  M3_resnest_lstm()


#########################################
device = "cuda:1"  

model_pure_name= 'LFB-resnet-30'
 # 'resnet/sl10_lr5e-5_6-25train/' cuda:0
LFB_path = "/media/yilin/tmrnet/models/temp/resnet/sl30_lr1e-5_6-25train/latest_model_16.pth"

# test M1
test_model(MODEL, test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000


/tmp/ipykernel_32773/935845069.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(10, device='cuda:1')
test_corrects: tensor(15, device='cuda:1')
test_corrects: tensor(15, device='cuda:1')
test_corrects: tensor(21, device='cuda:1')
test_corrects: tensor(27, device='cuda:1')
test_corrects: tensor(27, device='cuda:1')
test_corrects: tensor(34, device='cuda:1')
test_corrects: tensor(42, device='cuda:1')
test_corrects: tensor(50, device='cuda:1')
test_corrects: tensor(53, device='cuda:1')
test_corrects: tensor(55, device='cuda:1')
test_corrects: tensor(56, device='cuda:1')
test_corrects: tensor(58, device='cuda:1')
test_corrects: tensor(58, device='cuda:1')
test_corrects: tensor(58, device='cuda:1')
test_corrects: tensor(58, device='cuda:1')
test_corrects: tensor(59, device='cuda:1')
test_corrects: tensor(59, device='cuda:1')
test_corrects: tensor(62, device='cuda:1')
test_corrects: tensor(72, device='cuda:1')
test_corrects: tensor(82, device='cuda:1')
test_corrects: tensor(92, device='cuda:1')
test_corrects: tensor(102, device='cuda:1')
test_corre

test_corrects: tensor(797, device='cuda:1')
test_corrects: tensor(802, device='cuda:1')
test_corrects: tensor(807, device='cuda:1')
test_corrects: tensor(807, device='cuda:1')
test_corrects: tensor(813, device='cuda:1')
test_corrects: tensor(823, device='cuda:1')
test_corrects: tensor(833, device='cuda:1')
test_corrects: tensor(833, device='cuda:1')
test_corrects: tensor(837, device='cuda:1')
test_corrects: tensor(842, device='cuda:1')
test_corrects: tensor(851, device='cuda:1')
test_corrects: tensor(851, device='cuda:1')
test_corrects: tensor(861, device='cuda:1')
test_corrects: tensor(871, device='cuda:1')
test_corrects: tensor(881, device='cuda:1')
test_corrects: tensor(891, device='cuda:1')
test_corrects: tensor(901, device='cuda:1')
test_corrects: tensor(911, device='cuda:1')
test_corrects: tensor(921, device='cuda:1')
test_corrects: tensor(931, device='cuda:1')
test_corrects: tensor(941, device='cuda:1')
test_corrects: tensor(951, device='cuda:1')
test_corrects: tensor(958, devic

test_corrects: tensor(2170, device='cuda:1')
test_corrects: tensor(2179, device='cuda:1')
test_corrects: tensor(2189, device='cuda:1')
test_corrects: tensor(2199, device='cuda:1')
test_corrects: tensor(2208, device='cuda:1')
test_corrects: tensor(2218, device='cuda:1')
test_corrects: tensor(2228, device='cuda:1')
test_corrects: tensor(2238, device='cuda:1')
test_corrects: tensor(2248, device='cuda:1')
test_corrects: tensor(2258, device='cuda:1')
test_corrects: tensor(2265, device='cuda:1')
test_corrects: tensor(2275, device='cuda:1')
test_corrects: tensor(2285, device='cuda:1')
test_corrects: tensor(2295, device='cuda:1')
test_corrects: tensor(2305, device='cuda:1')
test_corrects: tensor(2314, device='cuda:1')
test_corrects: tensor(2320, device='cuda:1')
test_corrects: tensor(2328, device='cuda:1')
test_corrects: tensor(2337, device='cuda:1')
test_corrects: tensor(2346, device='cuda:1')
test_corrects: tensor(2354, device='cuda:1')
test_corrects: tensor(2364, device='cuda:1')
test_corre

...

test_corrects: tensor(4935, device='cuda:1')
test_corrects: tensor(4945, device='cuda:1')
test_corrects: tensor(4955, device='cuda:1')
test_corrects: tensor(4965, device='cuda:1')
test_corrects: tensor(4974, device='cuda:1')
test_corrects: tensor(4979, device='cuda:1')
test_corrects: tensor(4986, device='cuda:1')
test_corrects: tensor(4996, device='cuda:1')
test_corrects: tensor(5006, device='cuda:1')
test_corrects: tensor(5016, device='cuda:1')
test_corrects: tensor(5026, device='cuda:1')
test_corrects: tensor(5036, device='cuda:1')
test_corrects: tensor(5046, device='cuda:1')
test_corrects: tensor(5056, device='cuda:1')
test_corrects: tensor(5066, device='cuda:1')
test_corrects: tensor(5075, device='cuda:1')
test_corrects: tensor(5084, device='cuda:1')
test_corrects: tensor(5093, device='cuda:1')
test_corrects: tensor(5102, device='cuda:1')
test_corrects: tensor(5112, device='cuda:1')
test_corrects: tensor(5116, device='cuda:1')
test_corrects: tensor(5120, device='cuda:1')
test_corre

test_corrects: tensor(6421, device='cuda:1')
test_corrects: tensor(6431, device='cuda:1')
test_corrects: tensor(6441, device='cuda:1')
test_corrects: tensor(6451, device='cuda:1')
test_corrects: tensor(6461, device='cuda:1')
test_corrects: tensor(6470, device='cuda:1')
test_corrects: tensor(6480, device='cuda:1')
test_corrects: tensor(6486, device='cuda:1')
test_corrects: tensor(6495, device='cuda:1')
test_corrects: tensor(6501, device='cuda:1')
test_corrects: tensor(6511, device='cuda:1')
test_corrects: tensor(6521, device='cuda:1')
test_corrects: tensor(6531, device='cuda:1')
test_corrects: tensor(6541, device='cuda:1')
test_corrects: tensor(6551, device='cuda:1')
test_corrects: tensor(6553, device='cuda:1')
test_corrects: tensor(6562, device='cuda:1')
test_corrects: tensor(6572, device='cuda:1')
test_corrects: tensor(6581, device='cuda:1')
test_corrects: tensor(6590, device='cuda:1')
test_corrects: tensor(6594, device='cuda:1')
test_corrects: tensor(6603, device='cuda:1')
test_corre

test_corrects: tensor(7764, device='cuda:1')
test_corrects: tensor(7764, device='cuda:1')
test_corrects: tensor(7764, device='cuda:1')
test_corrects: tensor(7765, device='cuda:1')
test_corrects: tensor(7765, device='cuda:1')
test_corrects: tensor(7775, device='cuda:1')
test_corrects: tensor(7785, device='cuda:1')
test_corrects: tensor(7795, device='cuda:1')
test_corrects: tensor(7804, device='cuda:1')
test_corrects: tensor(7811, device='cuda:1')
test_corrects: tensor(7821, device='cuda:1')
test_corrects: tensor(7831, device='cuda:1')
test_corrects: tensor(7839, device='cuda:1')
test_corrects: tensor(7849, device='cuda:1')
test_corrects: tensor(7859, device='cuda:1')
test_corrects: tensor(7866, device='cuda:1')
test_corrects: tensor(7873, device='cuda:1')
test_corrects: tensor(7883, device='cuda:1')
test_corrects: tensor(7893, device='cuda:1')
test_corrects: tensor(7903, device='cuda:1')
test_corrects: tensor(7913, device='cuda:1')
test_corrects: tensor(7923, device='cuda:1')
test_corre

test_corrects: tensor(8797, device='cuda:1')
test_corrects: tensor(8807, device='cuda:1')
test_corrects: tensor(8817, device='cuda:1')
test_corrects: tensor(8826, device='cuda:1')
test_corrects: tensor(8836, device='cuda:1')
test_corrects: tensor(8846, device='cuda:1')
test_corrects: tensor(8854, device='cuda:1')
test_corrects: tensor(8859, device='cuda:1')
test_corrects: tensor(8867, device='cuda:1')
test_corrects: tensor(8873, device='cuda:1')
test_corrects: tensor(8873, device='cuda:1')
test_corrects: tensor(8873, device='cuda:1')
test_corrects: tensor(8874, device='cuda:1')
test_corrects: tensor(8883, device='cuda:1')
test_corrects: tensor(8893, device='cuda:1')
test_corrects: tensor(8902, device='cuda:1')
test_corrects: tensor(8912, device='cuda:1')
test_corrects: tensor(8922, device='cuda:1')
test_corrects: tensor(8932, device='cuda:1')
test_corrects: tensor(8940, device='cuda:1')
test_corrects: tensor(8942, device='cuda:1')
test_corrects: tensor(8942, device='cuda:1')
test_corre

test_corrects: tensor(10280, device='cuda:1')
test_corrects: tensor(10284, device='cuda:1')
test_corrects: tensor(10294, device='cuda:1')
test_corrects: tensor(10304, device='cuda:1')
test_corrects: tensor(10313, device='cuda:1')
test_corrects: tensor(10323, device='cuda:1')
test_corrects: tensor(10333, device='cuda:1')
test_corrects: tensor(10343, device='cuda:1')
test_corrects: tensor(10353, device='cuda:1')
test_corrects: tensor(10363, device='cuda:1')
test_corrects: tensor(10368, device='cuda:1')
test_corrects: tensor(10371, device='cuda:1')
test_corrects: tensor(10376, device='cuda:1')
test_corrects: tensor(10386, device='cuda:1')
test_corrects: tensor(10396, device='cuda:1')
test_corrects: tensor(10405, device='cuda:1')
test_corrects: tensor(10415, device='cuda:1')
test_corrects: tensor(10424, device='cuda:1')
test_corrects: tensor(10434, device='cuda:1')
test_corrects: tensor(10444, device='cuda:1')
test_corrects: tensor(10454, device='cuda:1')
test_corrects: tensor(10464, devic

In [11]:

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2   
MODEL = M1_resnet_lstm() # M2_densenet_lstm() /  M3_resnest_lstm()


#########################################
device = "cuda:1"  

model_pure_name= 'LFB-resnet-30'
 # 'resnet/sl10_lr5e-5_6-25train/' cuda:0
LFB_path = "/media/yilin/tmrnet/models/temp/resnet/sl30_lr1e-5_6-25train_2/latest_model_8.pth"

# test M1
test_model(MODEL, test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000


/tmp/ipykernel_8406/935845069.py:83: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(10, device='cuda:1')
test_corrects: tensor(15, device='cuda:1')
test_corrects: tensor(15, device='cuda:1')
test_corrects: tensor(19, device='cuda:1')
test_corrects: tensor(25, device='cuda:1')
test_corrects: tensor(32, device='cuda:1')
test_corrects: tensor(42, device='cuda:1')
test_corrects: tensor(52, device='cuda:1')
test_corrects: tensor(61, device='cuda:1')
test_corrects: tensor(68, device='cuda:1')
test_corrects: tensor(71, device='cuda:1')
test_corrects: tensor(72, device='cuda:1')
test_corrects: tensor(72, device='cuda:1')
test_corrects: tensor(72, device='cuda:1')
test_corrects: tensor(72, device='cuda:1')
test_corrects: tensor(72, device='cuda:1')
test_corrects: tensor(72, device='cuda:1')
test_corrects: tensor(73, device='cuda:1')
test_corrects: tensor(73, device='cuda:1')
test_corrects: tensor(73, device='cuda:1')
test_corrects: tensor(74, device='cuda:1')
test_corrects: tensor(84, device='cuda:1')
test_corrects: tensor(94, device='cuda:1')
test_correc

test_corrects: tensor(824, device='cuda:1')
test_corrects: tensor(824, device='cuda:1')
test_corrects: tensor(824, device='cuda:1')
test_corrects: tensor(824, device='cuda:1')
test_corrects: tensor(829, device='cuda:1')
test_corrects: tensor(839, device='cuda:1')
test_corrects: tensor(846, device='cuda:1')
test_corrects: tensor(846, device='cuda:1')
test_corrects: tensor(851, device='cuda:1')
test_corrects: tensor(856, device='cuda:1')
test_corrects: tensor(864, device='cuda:1')
test_corrects: tensor(864, device='cuda:1')
test_corrects: tensor(874, device='cuda:1')
test_corrects: tensor(884, device='cuda:1')
test_corrects: tensor(894, device='cuda:1')
test_corrects: tensor(904, device='cuda:1')
test_corrects: tensor(914, device='cuda:1')
test_corrects: tensor(922, device='cuda:1')
test_corrects: tensor(931, device='cuda:1')
test_corrects: tensor(941, device='cuda:1')
test_corrects: tensor(950, device='cuda:1')
test_corrects: tensor(952, device='cuda:1')
test_corrects: tensor(955, devic

test_corrects: tensor(2127, device='cuda:1')
test_corrects: tensor(2135, device='cuda:1')
test_corrects: tensor(2142, device='cuda:1')
test_corrects: tensor(2150, device='cuda:1')
test_corrects: tensor(2158, device='cuda:1')
test_corrects: tensor(2168, device='cuda:1')
test_corrects: tensor(2178, device='cuda:1')
test_corrects: tensor(2188, device='cuda:1')
test_corrects: tensor(2198, device='cuda:1')
test_corrects: tensor(2208, device='cuda:1')
test_corrects: tensor(2213, device='cuda:1')
test_corrects: tensor(2222, device='cuda:1')
test_corrects: tensor(2232, device='cuda:1')
test_corrects: tensor(2242, device='cuda:1')
test_corrects: tensor(2252, device='cuda:1')
test_corrects: tensor(2261, device='cuda:1')
test_corrects: tensor(2269, device='cuda:1')
test_corrects: tensor(2277, device='cuda:1')
test_corrects: tensor(2286, device='cuda:1')
test_corrects: tensor(2296, device='cuda:1')
test_corrects: tensor(2305, device='cuda:1')
test_corrects: tensor(2315, device='cuda:1')
test_corre

...

test_corrects: tensor(4594, device='cuda:1')
test_corrects: tensor(4603, device='cuda:1')
test_corrects: tensor(4610, device='cuda:1')
test_corrects: tensor(4619, device='cuda:1')
test_corrects: tensor(4627, device='cuda:1')
test_corrects: tensor(4631, device='cuda:1')
test_corrects: tensor(4638, device='cuda:1')
test_corrects: tensor(4648, device='cuda:1')
test_corrects: tensor(4658, device='cuda:1')
test_corrects: tensor(4668, device='cuda:1')
test_corrects: tensor(4678, device='cuda:1')
test_corrects: tensor(4688, device='cuda:1')
test_corrects: tensor(4697, device='cuda:1')
test_corrects: tensor(4707, device='cuda:1')
test_corrects: tensor(4717, device='cuda:1')
test_corrects: tensor(4727, device='cuda:1')
test_corrects: tensor(4736, device='cuda:1')
test_corrects: tensor(4745, device='cuda:1')
test_corrects: tensor(4755, device='cuda:1')
test_corrects: tensor(4765, device='cuda:1')
test_corrects: tensor(4772, device='cuda:1')
test_corrects: tensor(4782, device='cuda:1')
test_corre

test_corrects: tensor(6039, device='cuda:1')
test_corrects: tensor(6049, device='cuda:1')
test_corrects: tensor(6059, device='cuda:1')
test_corrects: tensor(6063, device='cuda:1')
test_corrects: tensor(6073, device='cuda:1')
test_corrects: tensor(6080, device='cuda:1')
test_corrects: tensor(6088, device='cuda:1')
test_corrects: tensor(6092, device='cuda:1')
test_corrects: tensor(6092, device='cuda:1')
test_corrects: tensor(6096, device='cuda:1')
test_corrects: tensor(6102, device='cuda:1')
test_corrects: tensor(6112, device='cuda:1')
test_corrects: tensor(6122, device='cuda:1')
test_corrects: tensor(6132, device='cuda:1')
test_corrects: tensor(6142, device='cuda:1')
test_corrects: tensor(6145, device='cuda:1')
test_corrects: tensor(6153, device='cuda:1')
test_corrects: tensor(6163, device='cuda:1')
test_corrects: tensor(6173, device='cuda:1')
test_corrects: tensor(6182, device='cuda:1')
test_corrects: tensor(6184, device='cuda:1')
test_corrects: tensor(6193, device='cuda:1')
test_corre

test_corrects: tensor(7305, device='cuda:1')
test_corrects: tensor(7306, device='cuda:1')
test_corrects: tensor(7306, device='cuda:1')
test_corrects: tensor(7310, device='cuda:1')
test_corrects: tensor(7320, device='cuda:1')
test_corrects: tensor(7330, device='cuda:1')
test_corrects: tensor(7340, device='cuda:1')
test_corrects: tensor(7350, device='cuda:1')
test_corrects: tensor(7359, device='cuda:1')
test_corrects: tensor(7368, device='cuda:1')
test_corrects: tensor(7378, device='cuda:1')
test_corrects: tensor(7388, device='cuda:1')
test_corrects: tensor(7396, device='cuda:1')
test_corrects: tensor(7406, device='cuda:1')
test_corrects: tensor(7416, device='cuda:1')
test_corrects: tensor(7417, device='cuda:1')
test_corrects: tensor(7424, device='cuda:1')
test_corrects: tensor(7434, device='cuda:1')
test_corrects: tensor(7444, device='cuda:1')
test_corrects: tensor(7454, device='cuda:1')
test_corrects: tensor(7464, device='cuda:1')
test_corrects: tensor(7474, device='cuda:1')
test_corre

test_corrects: tensor(8296, device='cuda:1')
test_corrects: tensor(8305, device='cuda:1')
test_corrects: tensor(8315, device='cuda:1')
test_corrects: tensor(8324, device='cuda:1')
test_corrects: tensor(8334, device='cuda:1')
test_corrects: tensor(8344, device='cuda:1')
test_corrects: tensor(8352, device='cuda:1')
test_corrects: tensor(8358, device='cuda:1')
test_corrects: tensor(8365, device='cuda:1')
test_corrects: tensor(8371, device='cuda:1')
test_corrects: tensor(8381, device='cuda:1')
test_corrects: tensor(8391, device='cuda:1')
test_corrects: tensor(8400, device='cuda:1')
test_corrects: tensor(8409, device='cuda:1')
test_corrects: tensor(8414, device='cuda:1')
test_corrects: tensor(8424, device='cuda:1')
test_corrects: tensor(8434, device='cuda:1')
test_corrects: tensor(8444, device='cuda:1')
test_corrects: tensor(8454, device='cuda:1')
test_corrects: tensor(8462, device='cuda:1')
test_corrects: tensor(8472, device='cuda:1')
test_corrects: tensor(8482, device='cuda:1')
test_corre

test_corrects: tensor(9833, device='cuda:1')
test_corrects: tensor(9835, device='cuda:1')
test_corrects: tensor(9840, device='cuda:1')
test_corrects: tensor(9850, device='cuda:1')
test_corrects: tensor(9860, device='cuda:1')
test_corrects: tensor(9870, device='cuda:1')
test_corrects: tensor(9880, device='cuda:1')
test_corrects: tensor(9887, device='cuda:1')
test_corrects: tensor(9891, device='cuda:1')
test_corrects: tensor(9894, device='cuda:1')
test_corrects: tensor(9894, device='cuda:1')
test_corrects: tensor(9902, device='cuda:1')
test_corrects: tensor(9907, device='cuda:1')
test_corrects: tensor(9909, device='cuda:1')
test_corrects: tensor(9919, device='cuda:1')
test_corrects: tensor(9920, device='cuda:1')
test_corrects: tensor(9920, device='cuda:1')
test_corrects: tensor(9929, device='cuda:1')
test_corrects: tensor(9938, device='cuda:1')
test_corrects: tensor(9946, device='cuda:1')
test_corrects: tensor(9956, device='cuda:1')
test_corrects: tensor(9966, device='cuda:1')
test_corre

# TESTM4 M7

In [56]:
# Long Term Feature bank
g_LFB_train = np.zeros(shape=(0, 512))
g_LFB_val = np.zeros(shape=(0, 512))

In [57]:
def get_long_feature(start_index_list, dict_start_idx_LFB, lfb):
    long_feature = []
    for j in range(len(start_index_list)):
        long_feature_each = []
        
        # 上一个存在feature的index
        last_LFB_index_no_empty = dict_start_idx_LFB[int(start_index_list[j])]
        
        # 
        for k in range(LFB_length):
            LFB_index = (start_index_list[j] - k - 1)
            if int(LFB_index) in dict_start_idx_LFB:                
                LFB_index = dict_start_idx_LFB[int(LFB_index)]
                long_feature_each.append(lfb[LFB_index])
                last_LFB_index_no_empty = LFB_index
            else:
                long_feature_each.append(lfb[last_LFB_index_no_empty])
            
        long_feature.append(long_feature_each)
    return long_feature

In [62]:
### m4 m7
g_LFB_test = np.zeros(shape=(0, 512))

def test_model(model,test_dataset, test_num_each):
    num_test = len(test_dataset)
    test_useful_start_idx = get_start_idx(sequence_length, test_num_each)
    test_useful_start_idx_LFB = get_start_idx(sequence_length, test_num_each)

    num_test_we_use = len(test_useful_start_idx)
    num_test_we_use_LFB = len(test_useful_start_idx_LFB)

    test_we_use_start_idx = test_useful_start_idx[0:num_test_we_use]
    test_we_use_start_idx_LFB = test_useful_start_idx_LFB[0:num_test_we_use_LFB]

    test_idx = []
    for i in range(num_test_we_use):
        for j in range(sequence_length):
            test_idx.append(test_we_use_start_idx[i] + j)

    num_test_all = len(test_idx)

    test_idx_LFB = []
    for i in range(num_test_we_use_LFB):
        for j in range(sequence_length):
            test_idx_LFB.append(test_we_use_start_idx_LFB[i] + j)

    dict_index, dict_value = zip(*list(enumerate(test_we_use_start_idx_LFB)))
    dict_test_start_idx_LFB = dict(zip(dict_value, dict_index))

    print('num test start idx : {:6d}'.format(len(test_useful_start_idx)))
    print('last idx test start: {:6d}'.format(test_useful_start_idx[-1]))
    print('num of test dataset: {:6d}'.format(num_test))
    print('num of test we use : {:6d}'.format(num_test_we_use))
    print('num of all test use: {:6d}'.format(num_test_all))
# TODO sampler

    test_loader = DataLoader(test_dataset,
                             batch_size=test_batch_size,
                             sampler=SequenceSampler(test_dataset, test_idx),
                             num_workers=workers)

    global g_LFB_test
    print("loading features!>.........")

    if not load_exist_LFB:
        g_LFB_test = np.zeros(shape=(0, 512))

        test_feature_loader = DataLoader(
            test_dataset,
            batch_size=test_batch_size,
            sampler=SequenceSampler(test_dataset, test_idx_LFB),
            num_workers=workers,
            pin_memory=False
        )

        ####################
        # model_LFB = resnet_lstm_LFB()

        model_LFB.load_state_dict(torch.load(LFB_path), strict=False)        
        ########################

        model_LFB.to(device)

        for params in model_LFB.parameters():
            params.requires_grad = False

        model_LFB.eval()

        with torch.no_grad():

            for data in test_feature_loader:

                inputs, labels_phase = data[0].to(device), data[1].to(device)
                

                inputs = inputs.view(-1, sequence_length, 3, 216,216)  #
                outputs_feature = model_LFB.forward(inputs)

                for j in range(len(outputs_feature)):
                    save_feature = outputs_feature.data.cpu()[j].numpy()
                    save_feature = save_feature.reshape(1, 512)
                    g_LFB_test = np.concatenate((g_LFB_test, save_feature), axis=0)

                print("test feature length:", len(g_LFB_test))

        print("finish!")
        g_LFB_test = np.array(g_LFB_test)
        
#         LFB_train_path_save_path = "./LFB/LFB_test_resnet.pkl"

        with open(LFB_test_path_save_path, 'wb') as f: 
            #"../Training TMRNet/LFB/g_LFB_test.pkl"
            # with open("./LFB/g_LFB_test.pkl", 'wb') as f:
            pickle.dump(g_LFB_test, f)

    else:
        with open(LFB_test_path_save_path, 'rb') as f: 
            g_LFB_test = pickle.load(f)

        print("load completed")

    print("g_LFB_test shape:", g_LFB_test.shape)    
    
        
    
    torch.cuda.empty_cache()
    
    ###########################3
    # model = resnet_lstm()
    #model.load_state_dict(torch.load(model_path))
    model.load_state_dict(torch.load(model_path))
    
    ###############################
    

    model.to(device)

    criterion = WeightedCrossEntropy()   #size_average=False

    model.eval()
    test_loss = 0.0
    test_corrects = 0
    test_start_time = time.time()

    all_preds = []
    all_preds_score = []


    with torch.no_grad():

        for data in test_loader:
            
            torch.cuda.empty_cache()            


            inputs, labels = data[0].to(device), data[1].to(device)
            labels = labels[(sequence_length - 1)::sequence_length]


            start_index_list = data[2]
            start_index_list = start_index_list[0::sequence_length]
            long_feature = get_long_feature(start_index_list=start_index_list,
                                            dict_start_idx_LFB=dict_test_start_idx_LFB,
                                            lfb=g_LFB_test)
            long_feature = torch.Tensor(long_feature).to(device)

            inputs = inputs.view(-1, sequence_length, 3, 216,216)

            outputs = model.forward(inputs, long_feature=long_feature)

            # softmax
            Sm = nn.Softmax()
            outputs = Sm(outputs)
            possibility, preds = torch.max(outputs.data, 1)


            for i in range(len(preds)):
                all_preds.append(preds[i].data.cpu())
            for i in range(len(possibility)):
                all_preds_score.append(possibility[i].data.cpu())
            
            
            loss = criterion(outputs, labels)


            test_loss += loss.data.item()
            test_corrects += torch.sum(preds == labels.data)
            print("test_corrects:",test_corrects)

    test_elapsed_time = time.time() - test_start_time
    test_accuracy = float(test_corrects) / float(num_test_we_use)
    test_average_loss = test_loss / num_test_we_use

    print('type of all_preds:', type(all_preds))
    print('leng of all preds:', len(all_preds))
    save_test = int("{:4.0f}".format(test_accuracy * 10000))
    pred_name = "./test_results_pkl/"+model_pure_name + '_test_' + str(save_test)  + '.pkl'
    

    with open(pred_name, 'wb') as f:
        pickle.dump(all_preds, f)

    print('test elapsed: {:2.0f}m{:2.0f}s'
          ' test loss: {:4.4f}'
          ' test accu: {:.4f}'
          .format(test_elapsed_time // 60,
                  test_elapsed_time % 60,
                  test_average_loss, test_accuracy))



In [45]:
test_dataset, test_num_each = get_test_data('../gen_datasets/test_paths_labels.pkl')

test_paths   :  15425
test_labels  :  15425


In [48]:
len(test_num_each)

25

In [63]:
# the first time to test, it is necessary to load feature bank

# example output of 2 test videos, 


sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2
LFB_length = 30
load_exist_LFB = False # False

MODEL = M4_resnet_lstm_nl() # M5_densenet_lstm_nl() /  M6_resnest_lstm_nl()


model_LFB = LFB_resnet_lstm()
# long feature bank path
LFB_test_path_save_path = "./LFB/LFB_test_resnet.pkl"




#########################################
device = "cuda:2"  
model_pure_name= 'NL-resnet' # used for the name of result pkl file
# test model path
model_path = "/media/yilin/pre-trained models/CataRCNet_models/resnet_no-local-only_M4.pth"

# lfb model path
LFB_path = "/media/yilin/pre-trained models/LFB_model/LFB_resnet_sl10_M1.pth"
###########################################


# train M4  # just example output using test01-test02
test_model(MODEL,test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000
loading features!>.........
test feature length: 10
test feature length: 20
test feature length: 30
test feature length: 40
test feature length: 50
test feature length: 60
test feature length: 70
test feature length: 80
test feature length: 90
test feature length: 100
test feature length: 110
test feature length: 120
test feature length: 130
test feature length: 140
test feature length: 150
test feature length: 160
test feature length: 170
test feature length: 180
test feature length: 190
test feature length: 200
test feature length: 210
test feature length: 220
test feature length: 230
test feature length: 240
test feature length: 250
test feature length: 260
test feature length: 270
test feature length: 280
test feature length: 290
test feature length: 300
test feature length: 310
test feature length: 320
test feature length: 330
test feature len

test feature length: 3140
test feature length: 3150
test feature length: 3160
test feature length: 3170
test feature length: 3180
test feature length: 3190
test feature length: 3200
test feature length: 3210
test feature length: 3220
test feature length: 3230
test feature length: 3240
test feature length: 3250
test feature length: 3260
test feature length: 3270
test feature length: 3280
test feature length: 3290
test feature length: 3300
test feature length: 3310
test feature length: 3320
test feature length: 3330
test feature length: 3340
test feature length: 3350
test feature length: 3360
test feature length: 3370
test feature length: 3380
test feature length: 3390
test feature length: 3400
test feature length: 3410
test feature length: 3420
test feature length: 3430
test feature length: 3440
test feature length: 3450
test feature length: 3460
test feature length: 3470
test feature length: 3480
test feature length: 3490
test feature length: 3500
test feature length: 3510
test feature

test feature length: 6310
test feature length: 6320
test feature length: 6330
test feature length: 6340
test feature length: 6350
test feature length: 6360
test feature length: 6370
test feature length: 6380
test feature length: 6390
test feature length: 6400
test feature length: 6410
test feature length: 6420
test feature length: 6430
test feature length: 6440
test feature length: 6450
test feature length: 6460
test feature length: 6470
test feature length: 6480
test feature length: 6490
test feature length: 6500
test feature length: 6510
test feature length: 6520
test feature length: 6530
test feature length: 6540
test feature length: 6550
test feature length: 6560
test feature length: 6570
test feature length: 6580
test feature length: 6590
test feature length: 6600
test feature length: 6610
test feature length: 6620
test feature length: 6630
test feature length: 6640
test feature length: 6650
test feature length: 6660
test feature length: 6670
test feature length: 6680
test feature

test feature length: 9470
test feature length: 9480
test feature length: 9490
test feature length: 9500
test feature length: 9510
test feature length: 9520
test feature length: 9530
test feature length: 9540
test feature length: 9550
test feature length: 9560
test feature length: 9570
test feature length: 9580
test feature length: 9590
test feature length: 9600
test feature length: 9610
test feature length: 9620
test feature length: 9630
test feature length: 9640
test feature length: 9650
test feature length: 9660
test feature length: 9670
test feature length: 9680
test feature length: 9690
test feature length: 9700
test feature length: 9710
test feature length: 9720
test feature length: 9730
test feature length: 9740
test feature length: 9750
test feature length: 9760
test feature length: 9770
test feature length: 9780
test feature length: 9790
test feature length: 9800
test feature length: 9810
test feature length: 9820
test feature length: 9830
test feature length: 9840
test feature

test feature length: 12530
test feature length: 12540
test feature length: 12550
test feature length: 12560
test feature length: 12570
test feature length: 12580
test feature length: 12590
test feature length: 12600
test feature length: 12610
test feature length: 12620
test feature length: 12630
test feature length: 12640
test feature length: 12650
test feature length: 12660
test feature length: 12670
test feature length: 12680
test feature length: 12690
test feature length: 12700
test feature length: 12710
test feature length: 12720
test feature length: 12730
test feature length: 12740
test feature length: 12750
test feature length: 12760
test feature length: 12770
test feature length: 12780
test feature length: 12790
test feature length: 12800
test feature length: 12810
test feature length: 12820
test feature length: 12830
test feature length: 12840
test feature length: 12850
test feature length: 12860
test feature length: 12870
test feature length: 12880
test feature length: 12890
t

...

test_corrects: tensor(5205, device='cuda:2')
test_corrects: tensor(5205, device='cuda:2')
test_corrects: tensor(5209, device='cuda:2')
test_corrects: tensor(5214, device='cuda:2')
test_corrects: tensor(5223, device='cuda:2')
test_corrects: tensor(5228, device='cuda:2')
test_corrects: tensor(5236, device='cuda:2')
test_corrects: tensor(5246, device='cuda:2')
test_corrects: tensor(5256, device='cuda:2')
test_corrects: tensor(5266, device='cuda:2')
test_corrects: tensor(5276, device='cuda:2')
test_corrects: tensor(5286, device='cuda:2')
test_corrects: tensor(5296, device='cuda:2')
test_corrects: tensor(5306, device='cuda:2')
test_corrects: tensor(5316, device='cuda:2')
test_corrects: tensor(5325, device='cuda:2')
test_corrects: tensor(5334, device='cuda:2')
test_corrects: tensor(5342, device='cuda:2')
test_corrects: tensor(5352, device='cuda:2')
test_corrects: tensor(5362, device='cuda:2')
test_corrects: tensor(5372, device='cuda:2')
test_corrects: tensor(5380, device='cuda:2')
test_corre

test_corrects: tensor(6677, device='cuda:2')
test_corrects: tensor(6687, device='cuda:2')
test_corrects: tensor(6697, device='cuda:2')
test_corrects: tensor(6707, device='cuda:2')
test_corrects: tensor(6717, device='cuda:2')
test_corrects: tensor(6726, device='cuda:2')
test_corrects: tensor(6735, device='cuda:2')
test_corrects: tensor(6745, device='cuda:2')
test_corrects: tensor(6755, device='cuda:2')
test_corrects: tensor(6765, device='cuda:2')
test_corrects: tensor(6775, device='cuda:2')
test_corrects: tensor(6785, device='cuda:2')
test_corrects: tensor(6795, device='cuda:2')
test_corrects: tensor(6805, device='cuda:2')
test_corrects: tensor(6815, device='cuda:2')
test_corrects: tensor(6825, device='cuda:2')
test_corrects: tensor(6835, device='cuda:2')
test_corrects: tensor(6845, device='cuda:2')
test_corrects: tensor(6846, device='cuda:2')
test_corrects: tensor(6854, device='cuda:2')
test_corrects: tensor(6856, device='cuda:2')
test_corrects: tensor(6865, device='cuda:2')
test_corre

test_corrects: tensor(8129, device='cuda:2')
test_corrects: tensor(8129, device='cuda:2')
test_corrects: tensor(8129, device='cuda:2')
test_corrects: tensor(8135, device='cuda:2')
test_corrects: tensor(8145, device='cuda:2')
test_corrects: tensor(8155, device='cuda:2')
test_corrects: tensor(8165, device='cuda:2')
test_corrects: tensor(8175, device='cuda:2')
test_corrects: tensor(8185, device='cuda:2')
test_corrects: tensor(8194, device='cuda:2')
test_corrects: tensor(8204, device='cuda:2')
test_corrects: tensor(8213, device='cuda:2')
test_corrects: tensor(8219, device='cuda:2')
test_corrects: tensor(8228, device='cuda:2')
test_corrects: tensor(8237, device='cuda:2')
test_corrects: tensor(8242, device='cuda:2')
test_corrects: tensor(8251, device='cuda:2')
test_corrects: tensor(8260, device='cuda:2')
test_corrects: tensor(8270, device='cuda:2')
test_corrects: tensor(8279, device='cuda:2')
test_corrects: tensor(8289, device='cuda:2')
test_corrects: tensor(8299, device='cuda:2')
test_corre

test_corrects: tensor(9401, device='cuda:2')
test_corrects: tensor(9410, device='cuda:2')
test_corrects: tensor(9420, device='cuda:2')
test_corrects: tensor(9430, device='cuda:2')
test_corrects: tensor(9440, device='cuda:2')
test_corrects: tensor(9450, device='cuda:2')
test_corrects: tensor(9459, device='cuda:2')
test_corrects: tensor(9465, device='cuda:2')
test_corrects: tensor(9468, device='cuda:2')
test_corrects: tensor(9474, device='cuda:2')
test_corrects: tensor(9484, device='cuda:2')
test_corrects: tensor(9494, device='cuda:2')
test_corrects: tensor(9504, device='cuda:2')
test_corrects: tensor(9513, device='cuda:2')
test_corrects: tensor(9515, device='cuda:2')
test_corrects: tensor(9525, device='cuda:2')
test_corrects: tensor(9535, device='cuda:2')
test_corrects: tensor(9545, device='cuda:2')
test_corrects: tensor(9555, device='cuda:2')
test_corrects: tensor(9563, device='cuda:2')
test_corrects: tensor(9569, device='cuda:2')
test_corrects: tensor(9571, device='cuda:2')
test_corre

test_corrects: tensor(10950, device='cuda:2')
test_corrects: tensor(10958, device='cuda:2')
test_corrects: tensor(10962, device='cuda:2')
test_corrects: tensor(10967, device='cuda:2')
test_corrects: tensor(10976, device='cuda:2')
test_corrects: tensor(10977, device='cuda:2')
test_corrects: tensor(10984, device='cuda:2')
test_corrects: tensor(10994, device='cuda:2')
test_corrects: tensor(11004, device='cuda:2')
test_corrects: tensor(11013, device='cuda:2')
test_corrects: tensor(11023, device='cuda:2')
test_corrects: tensor(11033, device='cuda:2')
test_corrects: tensor(11037, device='cuda:2')
test_corrects: tensor(11043, device='cuda:2')
test_corrects: tensor(11052, device='cuda:2')
test_corrects: tensor(11062, device='cuda:2')
test_corrects: tensor(11072, device='cuda:2')
test_corrects: tensor(11082, device='cuda:2')
test_corrects: tensor(11092, device='cuda:2')
test_corrects: tensor(11102, device='cuda:2')
test_corrects: tensor(11112, device='cuda:2')
test_corrects: tensor(11122, devic

In [64]:
# the first time to test, it is necessary to load feature bank

# example output of 2 test videos, 
# if you cannot load the pretrained model, you can change the layer name self.cnn to self.share

sequence_length = 10    # the length of input clip 
test_batch_size = 100  # batch size 
workers = 2
LFB_length = 30
load_exist_LFB = True # False

MODEL = M7_resnet_lstm_nl_tvl() # M5_densenet_lstm_nl() /  M6_resnest_lstm_nl()


model_LFB = LFB_resnet_lstm()
# long feature bank path
LFB_test_path_save_path = "./LFB/LFB_test_resnet.pkl"




#########################################
device = "cuda:2"  
model_pure_name= 'NL-TVL-resnet' # used for the name of result pkl file
# test model path
model_path = "/media/yilin/pre-trained models/CataRCNet_models/resnet_tvl_nl_M7.pth"

# lfb model path
LFB_path = "/media/yilin/pre-trained models/LFB_model/LFB_resnet_sl10_M1.pth"
###########################################


# train M4  # just example output using test01-test02
test_model(MODEL,test_dataset, test_num_each)

num test start idx :  15200
last idx test start:  15415
num of test dataset:  15425
num of test we use :  15200
num of all test use: 152000
loading features!>.........
load completed
g_LFB_test shape: (15200, 512)


/tmp/ipykernel_32289/201459335.py:153: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = Sm(outputs)


test_corrects: tensor(10, device='cuda:2')
test_corrects: tensor(15, device='cuda:2')
test_corrects: tensor(15, device='cuda:2')
test_corrects: tensor(21, device='cuda:2')
test_corrects: tensor(27, device='cuda:2')
test_corrects: tensor(27, device='cuda:2')
test_corrects: tensor(35, device='cuda:2')
test_corrects: tensor(39, device='cuda:2')
test_corrects: tensor(43, device='cuda:2')
test_corrects: tensor(46, device='cuda:2')
test_corrects: tensor(48, device='cuda:2')
test_corrects: tensor(49, device='cuda:2')
test_corrects: tensor(51, device='cuda:2')
test_corrects: tensor(57, device='cuda:2')
test_corrects: tensor(57, device='cuda:2')
test_corrects: tensor(60, device='cuda:2')
test_corrects: tensor(67, device='cuda:2')
test_corrects: tensor(69, device='cuda:2')
test_corrects: tensor(69, device='cuda:2')
test_corrects: tensor(72, device='cuda:2')
test_corrects: tensor(77, device='cuda:2')
test_corrects: tensor(87, device='cuda:2')
test_corrects: tensor(97, device='cuda:2')
test_correc

test_corrects: tensor(959, device='cuda:2')
test_corrects: tensor(966, device='cuda:2')
test_corrects: tensor(971, device='cuda:2')
test_corrects: tensor(971, device='cuda:2')
test_corrects: tensor(977, device='cuda:2')
test_corrects: tensor(987, device='cuda:2')
test_corrects: tensor(996, device='cuda:2')
test_corrects: tensor(996, device='cuda:2')
test_corrects: tensor(996, device='cuda:2')
test_corrects: tensor(999, device='cuda:2')
test_corrects: tensor(1000, device='cuda:2')
test_corrects: tensor(1000, device='cuda:2')
test_corrects: tensor(1001, device='cuda:2')
test_corrects: tensor(1008, device='cuda:2')
test_corrects: tensor(1018, device='cuda:2')
test_corrects: tensor(1028, device='cuda:2')
test_corrects: tensor(1038, device='cuda:2')
test_corrects: tensor(1048, device='cuda:2')
test_corrects: tensor(1058, device='cuda:2')
test_corrects: tensor(1068, device='cuda:2')
test_corrects: tensor(1076, device='cuda:2')
test_corrects: tensor(1086, device='cuda:2')
test_corrects: tenso

test_corrects: tensor(2252, device='cuda:2')
test_corrects: tensor(2261, device='cuda:2')
test_corrects: tensor(2271, device='cuda:2')
test_corrects: tensor(2281, device='cuda:2')
test_corrects: tensor(2291, device='cuda:2')
test_corrects: tensor(2301, device='cuda:2')
test_corrects: tensor(2311, device='cuda:2')
test_corrects: tensor(2321, device='cuda:2')
test_corrects: tensor(2331, device='cuda:2')
test_corrects: tensor(2341, device='cuda:2')
test_corrects: tensor(2348, device='cuda:2')
test_corrects: tensor(2358, device='cuda:2')
test_corrects: tensor(2368, device='cuda:2')
test_corrects: tensor(2378, device='cuda:2')
test_corrects: tensor(2388, device='cuda:2')
test_corrects: tensor(2397, device='cuda:2')
test_corrects: tensor(2402, device='cuda:2')
test_corrects: tensor(2410, device='cuda:2')
test_corrects: tensor(2419, device='cuda:2')
test_corrects: tensor(2429, device='cuda:2')
test_corrects: tensor(2439, device='cuda:2')
test_corrects: tensor(2449, device='cuda:2')
test_corre

...

test_corrects: tensor(5022, device='cuda:2')
test_corrects: tensor(5031, device='cuda:2')
test_corrects: tensor(5040, device='cuda:2')
test_corrects: tensor(5047, device='cuda:2')
test_corrects: tensor(5057, device='cuda:2')
test_corrects: tensor(5063, device='cuda:2')
test_corrects: tensor(5071, device='cuda:2')
test_corrects: tensor(5081, device='cuda:2')
test_corrects: tensor(5091, device='cuda:2')
test_corrects: tensor(5101, device='cuda:2')
test_corrects: tensor(5111, device='cuda:2')
test_corrects: tensor(5121, device='cuda:2')
test_corrects: tensor(5131, device='cuda:2')
test_corrects: tensor(5139, device='cuda:2')
test_corrects: tensor(5148, device='cuda:2')
test_corrects: tensor(5158, device='cuda:2')
test_corrects: tensor(5167, device='cuda:2')
test_corrects: tensor(5169, device='cuda:2')
test_corrects: tensor(5179, device='cuda:2')
test_corrects: tensor(5189, device='cuda:2')
test_corrects: tensor(5199, device='cuda:2')
test_corrects: tensor(5207, device='cuda:2')
test_corre

test_corrects: tensor(6440, device='cuda:2')
test_corrects: tensor(6450, device='cuda:2')
test_corrects: tensor(6460, device='cuda:2')
test_corrects: tensor(6468, device='cuda:2')
test_corrects: tensor(6478, device='cuda:2')
test_corrects: tensor(6488, device='cuda:2')
test_corrects: tensor(6493, device='cuda:2')
test_corrects: tensor(6498, device='cuda:2')
test_corrects: tensor(6508, device='cuda:2')
test_corrects: tensor(6518, device='cuda:2')
test_corrects: tensor(6528, device='cuda:2')
test_corrects: tensor(6538, device='cuda:2')
test_corrects: tensor(6548, device='cuda:2')
test_corrects: tensor(6558, device='cuda:2')
test_corrects: tensor(6568, device='cuda:2')
test_corrects: tensor(6578, device='cuda:2')
test_corrects: tensor(6587, device='cuda:2')
test_corrects: tensor(6597, device='cuda:2')
test_corrects: tensor(6598, device='cuda:2')
test_corrects: tensor(6606, device='cuda:2')
test_corrects: tensor(6608, device='cuda:2')
test_corrects: tensor(6617, device='cuda:2')
test_corre

test_corrects: tensor(7884, device='cuda:2')
test_corrects: tensor(7884, device='cuda:2')
test_corrects: tensor(7884, device='cuda:2')
test_corrects: tensor(7890, device='cuda:2')
test_corrects: tensor(7900, device='cuda:2')
test_corrects: tensor(7910, device='cuda:2')
test_corrects: tensor(7920, device='cuda:2')
test_corrects: tensor(7929, device='cuda:2')
test_corrects: tensor(7934, device='cuda:2')
test_corrects: tensor(7943, device='cuda:2')
test_corrects: tensor(7953, device='cuda:2')
test_corrects: tensor(7963, device='cuda:2')
test_corrects: tensor(7972, device='cuda:2')
test_corrects: tensor(7982, device='cuda:2')
test_corrects: tensor(7991, device='cuda:2')
test_corrects: tensor(7997, device='cuda:2')
test_corrects: tensor(8007, device='cuda:2')
test_corrects: tensor(8017, device='cuda:2')
test_corrects: tensor(8027, device='cuda:2')
test_corrects: tensor(8037, device='cuda:2')
test_corrects: tensor(8047, device='cuda:2')
test_corrects: tensor(8057, device='cuda:2')
test_corre

test_corrects: tensor(9192, device='cuda:2')
test_corrects: tensor(9201, device='cuda:2')
test_corrects: tensor(9211, device='cuda:2')
test_corrects: tensor(9220, device='cuda:2')
test_corrects: tensor(9230, device='cuda:2')
test_corrects: tensor(9240, device='cuda:2')
test_corrects: tensor(9248, device='cuda:2')
test_corrects: tensor(9255, device='cuda:2')
test_corrects: tensor(9262, device='cuda:2')
test_corrects: tensor(9267, device='cuda:2')
test_corrects: tensor(9274, device='cuda:2')
test_corrects: tensor(9284, device='cuda:2')
test_corrects: tensor(9292, device='cuda:2')
test_corrects: tensor(9299, device='cuda:2')
test_corrects: tensor(9303, device='cuda:2')
test_corrects: tensor(9313, device='cuda:2')
test_corrects: tensor(9323, device='cuda:2')
test_corrects: tensor(9333, device='cuda:2')
test_corrects: tensor(9343, device='cuda:2')
test_corrects: tensor(9351, device='cuda:2')
test_corrects: tensor(9355, device='cuda:2')
test_corrects: tensor(9357, device='cuda:2')
test_corre

test_corrects: tensor(10724, device='cuda:2')
test_corrects: tensor(10730, device='cuda:2')
test_corrects: tensor(10738, device='cuda:2')
test_corrects: tensor(10738, device='cuda:2')
test_corrects: tensor(10739, device='cuda:2')
test_corrects: tensor(10747, device='cuda:2')
test_corrects: tensor(10756, device='cuda:2')
test_corrects: tensor(10766, device='cuda:2')
test_corrects: tensor(10775, device='cuda:2')
test_corrects: tensor(10784, device='cuda:2')
test_corrects: tensor(10789, device='cuda:2')
test_corrects: tensor(10794, device='cuda:2')
test_corrects: tensor(10800, device='cuda:2')
test_corrects: tensor(10810, device='cuda:2')
test_corrects: tensor(10820, device='cuda:2')
test_corrects: tensor(10830, device='cuda:2')
test_corrects: tensor(10839, device='cuda:2')
test_corrects: tensor(10845, device='cuda:2')
test_corrects: tensor(10854, device='cuda:2')
test_corrects: tensor(10863, device='cuda:2')
test_corrects: tensor(10871, device='cuda:2')
test_corrects: tensor(10878, devic